## Intro to EDA with Time Series 

> This notebook is a modified version of a notebook from the Berlin Time Series Meetup by Juan Orduz. If you want to dive deeper into time series we can highly reccomend to check out the repository of the meetup on [github](https://github.com/juanitorduz/btsa).

Time series forcasting is another interesting aspect of Data Science. In this notebook we will have a look at some simple plots and methods you can apply when you start with a time series project. We will show on a simple example how to manipulate and plot time series data in python.

#### At the end of this notebook you should:
 * know some helpful plots to start with your time series data
 * know how to decompose a time series and what the single components are

## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [ ]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}
)
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
sns_c

## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [ ]:
# Import data
raw_df = pd.read_csv('data/basel_weather.csv')
raw_df.head()

In [ ]:
raw_df.info()

## Format Data & Feature Engineering

The dataset includes a timestamp. It might be interesting to have a look at additional features like for example `month`, `day`, `day of the year`, and `hour`.

In [ ]:
# Create working copy of dataframe
data_df = raw_df.copy()

# Rename columns in a more pythonic way
data_df = data_df.rename(columns={
    'Basel Temperature [2 m elevation corrected]': 'temperature', 
    'Basel Precipitation Total': 'precipitation', 
    'Basel Wind Speed [10 m]': 'wind_speed', 
    'Basel Wind Direction [10 m]': 'wind_direction'
    }
)

# Convert timestamp to datetime object
# Extract additional features from timestamp column
data_df = data_df.assign(
    timestamp = lambda x: pd.to_datetime(x['timestamp']), 
    date = lambda x: x['timestamp'].dt.date,
    year = lambda x: x['timestamp'].dt.year,
    month = lambda x: x['timestamp'].dt.month,
    day = lambda x: x['timestamp'].dt.day,
    dayofyear = lambda x: x['timestamp'].dt.dayofyear,
    hour = lambda x: x['timestamp'].dt.hour,
)

data_df.head()

## Visualize the Data

Let us start by plotting the temperature data over time. We can either use the hourly development or aggregate the feature by day to get a less dense plot. 

In [ ]:
# Temperature hourly development over time 
fig, ax = plt.subplots()
sns.lineplot(x='timestamp', y='temperature', data=data_df, ax=ax)
ax.set(title='Basel - Temperature (Hourly)', ylabel=r'$^\circ$C');

fig.savefig("visualisations/Basel_Temp_hourly.png",dpi=300)

In [ ]:
# Aggregate temperature by day
daily_data_df = data_df \
    .groupby(['date', 'year', 'month', 'day', 'dayofyear'], as_index=False)\
    .agg({'temperature': np.mean}) \
    .set_index('date')

In [ ]:
# Plot temperature on daily basis 
fig, ax = plt.subplots()
sns.lineplot(x='date', y='temperature', data=daily_data_df.reset_index(), ax=ax)
ax.set(title='Basel - Temperature (Daily)', ylabel=r'$^\circ$C');

fig.savefig("visualisations/Basel_Temp_daily.png",dpi=300)

Both plots show a clear yearly seasonality but other effects like a trend for example are hard to recognize.

We can focus on the seasonality and create a plot to show the yearly seasonality of temperature and its variation:

In [ ]:
daily_data_df

In [ ]:
# Plot yearly seasonality
fig, ax = plt.subplots() 

pd.pivot_table(data=daily_data_df[['year', 'dayofyear', 'temperature']], index='dayofyear', columns='year') \
    ['temperature'] \
    .plot(cmap='viridis', alpha=0.5, ax=ax)

ax.legend(title='year', loc='center left', bbox_to_anchor=(1, 0.5))
ax.set(title='Basel - Yearly Temperature (Daily)', ylabel=r'$^\circ$C');

fig.savefig("visualisations/Basel_Yearly_Temperature.png",dpi=300)

We can also create a *polar plot* that depicts the seasonal change over the year in a circle. This way to visualize the change can be seen in the next plot. This is (sometimes) a nice way to depicture data that follow cyclical patterns (in this case: once around the circle represents one year). 

In [ ]:
# Polar plot for seasonality 
ax = plt.subplot(111, projection='polar')

# Convert and plot data
daily_data_df \
    .assign(day_of_year_cyclic = lambda x: x['dayofyear'].transform(lambda x: 2*np.pi*x/365.25)) \
    .pipe((sns.lineplot, 'data'), 
        x='day_of_year_cyclic', 
        y='temperature', 
        hue='month',
        palette=sns.color_palette("husl", 12),
        ax=ax
    )

fancy_plot=True     #this is only to make the plot more beautiful. If you just want to see how the data looks without the adjustments to the plot set fancy_plot to False
if(fancy_plot): 
    days_per_month=[0] + [monthrange(2021, i)[1] for i in range(1,12)]      #findout how many days each month has ()
    month_start=np.cumsum(days_per_month) +1                                #add 1 to start at 0 instead of 0, take the cumsum to get ech months starting day
    month_start_theta=[i *2 * np.pi / 365.25 for i in month_start]          #turn start day into an angle (in rad), use 365.25 as the average length of a year

    month_label=[month_name[i] for i in range(1,13)]
    month_label_long=[label+'\n(Day ' +str(month_start[ind]) +')' for ind,label in enumerate(month_label)]

    ax.set_title('Basel Temperature ', va='bottom',pad=22);
    ax.spines.clear()
    
    ax.set_xlabel('')
    ax.set_xticks(month_start_theta)
    ax.set_xticklabels(month_label_long)
    
    ax.set_ylabel('')    
    ax.set_ylim(-5,28)
    ax.set_yticks(yt:=[0,5,10,15,20])
    ax.set_yticklabels([str(t)+'°C' for t in yt], rotation = 45)

    #Arrows / Annotations
    style = "Simple, tail_width=0.5, head_width=4, head_length=8"
    kw = dict(arrowstyle=style, color="dimgrey")
    ax.set_rlabel_position(1) 
    ax.text(13*2*np.pi/360,24,"Days",size=14,color='dimgrey',rotation=-80,va='center')
    ax.text(-3*2*np.pi/360,16,"Temperature",size=14,color='dimgrey',rotation=-0,va='center')
    a1 = patches.FancyArrowPatch((1*np.pi/180, -5), (1*np.pi/180, 26), **kw)

    a2 = patches.FancyArrowPatch((1*np.pi/180, 24), (25*np.pi/180, 24),
                                connectionstyle=f"arc3,rad={0.105}", **kw)
    
    ax.add_patch(a1)
    ax.add_patch(a2)


    ax.set_rorigin(-5)
    ax.xaxis.set_tick_params(which='major',pad=10)

    ax.legend(labels=month_label,ncol=2,facecolor='white',edgecolor='white',bbox_to_anchor=(1.1, 1.1), loc=1)

    ax.figure.set_figwidth(12)
    ax.figure.set_figheight(12)


ax.figure.savefig("visualisations/Basel_Temp_polar.png",dpi=300)

## Smoothing

Usually the first question of data analysis is "Can we extract general, global pattern here?"
In time series with cyclic seasonal variation, this means that we want to smooth out this seasonality and let the global trend manifest itself.
Most classical, first go-to way for this smoothing is to use [Moving Average](https://en.wikipedia.org/wiki/Moving_average). Please also refer to the Notebook on moving averages for a better understanding.

In [ ]:
# Plot moving average of different length (week, month, year)
ma = [7, 30, 365]


smooth_daily_data_df = daily_data_df \
    .reset_index() \
    .assign(date = lambda x: x['date'].transform(pd.to_datetime))

# Smooth and plot
fig, ax = plt.subplots(len(ma)+1, 1, figsize=(12, 9), constrained_layout=True, sharex=True)
plt.suptitle('Basel Temperature (Daily) - Moving Average Smoothing', y=1.02);

for i, m in enumerate(ma):
    smooth_daily_data_df[f'temp_smooth_ma_{m}'] = smooth_daily_data_df['temperature'].rolling(window=m,center=True).mean() #compute the rolling mean

    sns.lineplot(x='date', y='temperature', label='Temperature (Signal)', data=smooth_daily_data_df,  ax=ax[i])
    sns.lineplot(x='date', y=f'temp_smooth_ma_{m}', label=f'Temperature smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=NF_ORANGE, ax=ax[i])

    ax[i].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
    ax[i].set(title='', ylabel=r'$^\circ$C');
        
sns.lineplot(x='date', y=f'temp_smooth_ma_{m}', label=f'Temperature smoothed:\n ma = {m} days', data=smooth_daily_data_df, color=sns_c[1], ax=ax[i+1])
ax[i+1].legend(title='', loc='center left', bbox_to_anchor=(1, 0.5))
ax[i+1].set(title='', ylabel=r'$^\circ$C');

    
fig.savefig("visualisations/Basel_Temp_MA_Smoothing.png",dpi=300)

## Time Series Decomposition

We can go deeper and decompose time series data into the different ingredients to understand what the sources of variation are we see in the data.
The method `seasonal_decompose` from `statsmodels.tsa.seasonal` can help us to decompose the daily data. This method is based on moving averages and returns the **trend**, **seasonality** and **residuals** of our timeseries.

The stpes inside of this function is:
* we first check for global trends using Moving average
* we de-trend our data by substituding the trend values from the original data
* then we estimate seasonal component by taking averages of each season's value - for example, the effect of January is the average of all de-trended January values in the data

We can define which seasonality we want to extract, lets pick a period of 365 Days, because this seems to make the most sense.

In [ ]:
# We use the parameter `period` = 365 to extract the yearly seasonality. 
seas_decomp_yearly = seasonal_decompose(
    x=daily_data_df['temperature'], 
    model='additive', 
    two_sided=True,
    period= 365)

# Plots:
fig, ax = plt.subplots(4, 1, figsize=(12, 12), constrained_layout=True)

#Plot Signal
ax[0].set(title='Observed data (signal)', 
          ylabel=r'$^\circ$C')
seas_decomp_yearly.observed.plot(color=sns_c[0], 
                              linewidth=1,
                              sharex=True,
                              ax=ax[0])
#Plot Trend
ax[1].set(title='Trend (364 days moving average)', 
          ylabel=r'$^\circ$C')
seas_decomp_yearly.trend.plot(color=sns_c[1], 
                              linewidth=1,
                              sharex=True,
                              ax=ax[1])
#Plot Seasonality
ax[2].set(title='Seasonality', 
          ylabel=r'$^\circ$C')
seas_decomp_yearly.seasonal.plot(
                                color=sns_c[2], 
                                linewidth=1,
                                sharex=True,
                                ax=ax[2])
#Plot residual
ax[3].set(title='Residual', 
          ylabel=r'$^\circ$C');
ax[3].scatter(
    x=seas_decomp_yearly.resid.index,
    y=seas_decomp_yearly.resid,
    color=sns_c[3],
    s=4)

for i in range(4):
    ax[i].set_xlim(pd.to_datetime("2008"),pd.to_datetime("2021"))
    
fig.savefig("visualisations/Basel_Temp_TSA_decomp.png",dpi=300)

As we can see, a simple call of ```seasonal_decompose()``` allows us to create a decomposition and access the trend, seasonality and residual!
We try to produce a nice looking plot, which might make this process seem complicated. Hence, please have a look at the following cell which contains the results as well:

In [ ]:
#Clutter-free version of the plot
#1. Compute
seas_decomp_yearly = seasonal_decompose(
    x=daily_data_df['temperature'], 
    model='additive', 
    two_sided=True,
    period= 365)

#2. Plot
fig= seas_decomp_yearly.plot()